In [1]:
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [11]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def windowed_dataset(series, batch_size, window_size = 64, shuffle_buffer = 1_000):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [3]:
data = pd.read_csv('./dataset/clean-sensors.csv', infer_datetime_format=True, header=0, index_col='datetime')

C:\Users\Bintang\AppData\Local\Temp\ipykernel_12108\3182949423.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('./dataset/clean-sensors.csv', infer_datetime_format=True, header=0, index_col='datetime')


In [4]:
data.head()

,Temperature,Humidity,Illumination,kat_temperature,kat_humidity,kat_illumination,Kipas,Humidifier,LED
datetime,,,,,,,,,
03/07/2023 0:00:00,28.0,75,76.61,2,2,3,0,0,1
03/07/2023 0:01:00,28.0,75,75.33,2,2,3,0,0,1
03/07/2023 0:02:00,28.0,75,74.49,2,2,3,0,0,1
03/07/2023 0:03:00,28.0,75,74.49,2,2,3,0,0,1
03/07/2023 0:04:00,28.0,75,74.49,2,2,3,0,0,1


In [5]:
data.columns

Index(['Temperature', 'Humidity', 'Illumination', 'kat_temperature',
       'kat_humidity', 'kat_illumination', 'Kipas', 'Humidifier', 'LED'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11507 entries, 03/07/2023 0:00:00 to 10/07/2023 23:59:00
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature       11507 non-null  float64
 1   Humidity          11507 non-null  int64  
 2   Illumination      11063 non-null  float64
 3   kat_temperature   11507 non-null  int64  
 4   kat_humidity      11507 non-null  int64  
 5   kat_illumination  11507 non-null  int64  
 6   Kipas             11507 non-null  int64  
 7   Humidifier        11507 non-null  int64  
 8   LED               11507 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 899.0+ KB


In [7]:
data_fan = data.drop(['Humidifier', 'LED'], axis=1)
data_humid = data.drop(['LED', 'Kipas'], axis=1)
data_led = data.drop(['Kipas', 'Humidifier'], axis=1)

In [8]:
n_data_fan = normalize_series(data_fan.copy(), data_fan.min(axis=0), data_fan.max(axis=0))
n_data_humid = normalize_series(data_humid.copy(), data_humid.min(axis=0), data_humid.max(axis=0))
n_data_led = normalize_series(data_led.copy(), data_led.min(axis=0), data_led.max(axis=0))

In [9]:
split_time = int(len(data) * 0.8)
print(split_time)

9205


In [10]:
train_fan = n_data_fan[:split_time]
valid_fan = n_data_fan[split_time:]

train_led = n_data_led[:split_time]
valid_led = n_data_led[split_time:]

train_humid = n_data_humid[:split_time]
valid_humid = n_data_humid[split_time:]

In [12]:
train_series_fan = windowed_dataset(train_fan, 256)
valid_series_fan = windowed_dataset(valid_fan, 256)

train_series_humid = windowed_dataset(train_humid, 256)
valid_series_humid = windowed_dataset(valid_humid, 256)

train_series_led = windowed_dataset(train_led, 256)
valid_series_led = windowed_dataset(valid_led, 256)

In [ ]:
model =